In [387]:
import requests
from bs4 import BeautifulSoup

In [456]:
#print(response.text) to see everything

response = requests.get("https://topwebfiction.com/?ranking=at")
story_titles = soup.findAll('span', attrs={"class":"title"})

titles = []
for row in story_titles:
    titles.append(row.text.strip())

In [389]:
titles[0:5]

['A Practical Guide to Evil',
 'Worm',
 'Ward',
 'The Wandering Inn',
 'Metaworld Chronicles']

In [390]:
story_authors = soup.findAll('span', attrs={"class":"byline"})

authors = []
for row in story_authors:
    authors.append(row.text.strip())

for x in range(len(authors)):
    authors[x] = re.sub('by ','',authors[x])

In [391]:
authors[0:5]

['ErraticErrata', 'Wildbow', 'wildbow', 'pirateaba', 'Wutosama']

In [ ]:
story_votes = soup.findAll('td', attrs={"class":"info"})
votes = []
for x in story_votes:
    votes.append(x.text)
votes2 = votes
for x in range(len(votes)):
    votes2[x] = re.sub(r'\n', '', str(votes[x]))
    votes2[x] = re.sub(r'\xa0boosters', '', str(votes2[x]))

In [437]:
votes2[0:5]

['47415', '39080', '24097', '22246', '19984']

In [457]:
story_tags = soup.findAll('p', attrs={"class":"tags"})
tags = []
for x in story_tags:
    tags.append(x.text)

In [459]:
clean_tags = tags
for x in range(len(clean_tags)):
    clean_tags[x] = re.sub(r'\\n', ' ', str(clean_tags[x]))
    clean_tags[x] = re.sub('\'', '', str(clean_tags[x]))
    clean_tags[x] = re.sub(', ', '-', str(clean_tags[x]))
    clean_tags[x] = re.split(' ', str(clean_tags[x]))
clean_tags[0:5]

[['[adventure',
  'anti-hero',
  'coming-of-age',
  'fantasy',
  'magic',
  'young-adult',
  ']'],
 ['[action',
  'adventure',
  'complete',
  'crime',
  'dark',
  'high-school',
  'modern-supernatural',
  'superhero',
  'teenagers',
  ']'],
 ['[action',
  'adventure',
  'alternate-universe',
  'crime',
  'dark',
  'drama',
  'post-apocalyptic',
  'superhero',
  'teenagers',
  'young-adult',
  ']'],
 ['[fantasy', 'magic', ']'],
 ['[fantasy', 'magic', 'relationships', 'young-adult', ']']]

In [383]:
for x in range(len(clean_tags)):
    clean_tags[x] = re.sub('\'', '', str(clean_tags[x]))
x[0]

"['[adventure', 'anti-hero', 'coming-of-age', 'fantasy', 'magic', 'young-adult', ']']"

In [384]:
x = re.sub('\'', '', str(x))
x

'[[adventure, anti-hero, coming-of-age, fantasy, magic, young-adult, ]]'

In [385]:
x = re.sub(', ', '-', str(x))
x

'[[adventure-anti-hero-coming-of-age-fantasy-magic-young-adult-]]'

In [ ]:
for x in range(len(clean_tage)):

In [292]:
tags2[0:5]

[['adventure\nanti-hero\ncoming',
  'of',
  'age\nfantasy\nmagic\nyoung',
  'adult\n'],
 ['action\nadventure\ncomplete\ncrime\ndark\nhigh',
  'school\nmodern',
  'supernatural\nsuperhero\nteenagers\n'],
 ['action\nadventure\nalternate',
  'universe\ncrime\ndark\ndrama\npost-apocalyptic\nsuperhero\nteenagers\nyoung',
  'adult\n'],
 ['fantasy\nmagic\n'],
 ['fantasy\nmagic\nrelationships\nyoung', 'adult\n']]

In [448]:
response = requests.get("https://topwebfiction.com/listings/a-practical-guide-to-evil/")
similar_stories_titles = soup.findAll('span', attrs={"class":"title"})

similar_titles = []
for row in similar_stories_titles:
    similar_titles.append(row.text.strip())

In [449]:
similar_titles

['A Practical Guide to Evil',
 'Worm',
 'Ward',
 'The Wandering Inn',
 'Metaworld Chronicles',
 'Twig',
 'Pact',
 'The Zombie Knight Saga',
 'The Gods are Bastards',
 'How To Avoid Death On A Daily Basis',
 'The Iron Teeth',
 'The Good Student',
 'The Legion of Nothing',
 'Inexorable Chaos',
 'Unsong',
 'Defiance of the Fall',
 'Star Child',
 'Everybody Loves Large Chests',
 'Citadel',
 'The Primal Hunter',
 'Savage Divinity',
 'Forge of Destiny',
 'Heretical Edge',
 'RE: Trailer Trash',
 'Into the Mire',
 'I was reincarnated as a Magic Academy!',
 'Salvos',
 '100 Luck and the Dragon Tamer Skill!',
 'Twisted Cogs',
 'Deeper Darker',
 'Anathema',
 'Katalepsis',
 'The Gam3',
 'Bitter',
 'The Divine Elements',
 'Brennus',
 'Six Chances',
 'Vigor Mortis',
 'Taint',
 'Beneath the Dragoneye Moons',
 'Aethernea',
 'I hate being wed in a fantasy world!',
 'End Online',
 'Void Domain',
 'The Bridge and Other Short Stories',
 'Two Worlds',
 'Summus Proelium',
 'Google',
 'Tower of Somnus',
 'A J

In [444]:
<div class="pure-u-1 pure-u-md-11-24">
      <h3>Has boosters in common</h3>
              <ul>
                      <li>
              <span class="title"><a href="https://topwebfiction.com/listings/the-gods-are-bastards/">The Gods are Bastards</a></span>
              <span class="byline">by D. D. Webb</span>
            </li>
                      <li>
              <span class="title"><a href="https://topwebfiction.com/listings/the-wandering-inn/">The Wandering Inn</a></span>
              <span class="byline">by pirateaba</span>
            </li>
                      <li>
              <span class="title"><a href="https://topwebfiction.com/listings/worm/">Worm</a></span>
              <span class="byline">by Wildbow</span>
            </li>
                      <li>
              <span class="title"><a href="https://topwebfiction.com/listings/ward/">Ward</a></span>
              <span class="byline">by wildbow</span>
            </li>
                      <li>
              <span class="title"><a href="https://topwebfiction.com/listings/metaworld-chronicles/">Metaworld Chronicles</a></span>
              <span class="byline">by Wutosama</span>
            </li>
                      <li>
              <span class="title"><a href="https://topwebfiction.com/listings/twig/">Twig</a></span>
              <span class="byline">by Wildbow</span>
            </li>
                      <li>
              <span class="title"><a href="https://topwebfiction.com/listings/the-zombie-knight/">The Zombie Knight Saga</a></span>
              <span class="byline">by George M. Frost</span>
            </li>
                      <li>
              <span class="title"><a href="https://topwebfiction.com/listings/the-iron-teeth/">The Iron Teeth</a></span>
              <span class="byline">by ClearMadness</span>
            </li>
                      <li>
              <span class="title"><a href="https://topwebfiction.com/listings/pact/">Pact</a></span>
              <span class="byline">by Wildbow</span>
            </li>
                      <li>
              <span class="title"><a href="https://topwebfiction.com/listings/heretical-edge/">Heretical Edge</a></span>
              <span class="byline">by Cerulean</span>
            </li>
                  </ul>
          </div>

<!DOCTYPE html>
<html lang="en-CA">
<head>
  <title>A Practical Guide to Evil | Top Web Fiction</title>

  <meta name="MobileOptimized" content="width">
  <meta name="HandheldFriendly" content="true">
  <meta name="viewport" content="width=device-width">
  
  <link rel="stylesheet" href="https://unpkg.com/purecss@2.0.3/build/pure-min.css" integrity="sha384-cg6SkqEOCV1NbJoCu11+bm0NvBRc8IYLRGXkmNrqUBfTjmMYwNKPWBTIKyw9mHNJ" crossorigin="anonymous">
  <link rel="stylesheet" href="https://unpkg.com/purecss@2.0.3/build/grids-responsive-min.css">
  <link href="https://fonts.googleapis.com/css2?family=Roboto:ital,wght@0,400;0,500;1,400;1,500&display=swap" rel="stylesheet">
  <link rel="stylesheet" href="/css/styles.css?v=1613204859">
  <link rel="stylesheet" href="/css/colours-default.css?v=1613450652">

  <script src="https://code.jquery.com/jquery-3.5.1.min.js" integrity="sha256-9/aliU8dGd2tb6OSsuzixeV4y/faTqgFtohetphbbj0=" crossorigin="anonymous"></script>

</head>
<body>
  <p class="skip-l